<a href="https://colab.research.google.com/github/Desmondonam/AWS-EC2-S3-and-DynamoDB-application/blob/main/496073516_LabSDT_Pt2_459044197352219.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To run the codes, ensure that you are connected to data that is provided, and the module named sdt_exp is working.

In [ ]:
!pip install pingouin


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 185 kB 34.0 MB/s 
     |████████████████████████████████| 9.9 MB 69.0 MB/s 
  Created wheel for pingouin: filename=pingouin-0.5.2-py3-none-any.whl size=196208 sha256=9d9a497fa2fc8ed44d74e6c42cf66bdecbf44e66e552a1b05b5f3955eb728a20
  Stored in directory: /root/.cache/pip/wheels/75/b0/89/f6866fc5accea9bec04f4c9f2a242520d6e8e7b8fcf89e139c
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7047 sha256=a54c41b6c0db970230a56743ebad874e7e441197f9d67494c807937c06a96521
  Stored in directory: /root/.cache/pip/wheels/6a/33/c4/0ef84d7f5568c2823e3d63a6e08988852fb9e4bc822034870a
Successfully built pingouin littleutils
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [ ]:
!pip install ipycanvas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 255 kB 29.4 MB/s 
     |████████████████████████████████| 1.6 MB 66.9 MB/s 


In [ ]:
!pip install sdt_exp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sdt_exp (from versions: none)
ERROR: No matching distribution found for sdt_exp


In [ ]:
from IPython.core.display import HTML, Markdown, display

import numpy as np
import numpy.random as npr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.formula.api as smf
import pingouin as pg
import math

import ipywidgets as widgets

import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import numpy as np
import pandas as pd
from datetime import datetime
from ipycanvas import Canvas, hold_canvas

#from sdt_exp import *

# Enable plots inside the Jupyter Notebook
%matplotlib inline

# Signal detection: Part 2

Authored by *Todd Gureckis*.

In the previous lab you learned some of the basic analysis steps involved in signal detection analysis.  In particular, we learned how to compute a statistic we called d' (dprime) and c (bias/criterion).  These variables are commonly used in studies of perception to understand the performance of a subject on a task independent of their response bias (here meaning tendency to give a particular response).  

In the second part of the lab we will analyze data from a simple detection experiment you will have just conducted on yourselves.  You will get to put your python knowledge into practice by reading in the data analyzing it.

## The Experiment Instructions

In this experiment you job will be similar to that of a doctor.  There are going to be small "clusters" of points which we might think of as a tumor or some other biological measurement.  You can use the interaction element below to examine a few of these "clusters".

Try using the widget below to slide between a lot of dots within a cluster and a small number.  Sweep back and forth and get an idea of what a "cluster" could look like.  The dots are spread out to a certain degree as you can see more clearly with the large number of dots.

In [ ]:
@widgets.interact(dots=(1,75, 5))
def draw(dots):
    return draw_stimulus(include_source=True, n_background_dots=0, n_source_dots=dots, source_sd=10, source_border=20, width=400, height=400)

interactive(children=(IntSlider(value=36, description='dots', max=75, min=1, step=5), Output()), _dom_classes=…

The challenge is that these dots are going to be hidden in a background of "noise" which is other dots unrelated to the cluster.  Here is an example of the background with no cluster present:

In [ ]:
draw_stimulus(include_source=False, n_background_dots=30*30, n_source_dots=0, source_sd=10, source_border=20, width=400, height=400)

NameError: ignored

You can run the above cell many times to see different example patterns when the cluster is absent.

Next here is an example of a very large cluster hidden in the background noise:

In [ ]:
draw_stimulus(include_source=True, n_background_dots=30*30, n_source_dots=60, source_sd=10, source_border=20, width=400, height=400)

Canvas(height=425, layout=Layout(grid_area='canvas'), width=425)

From this example you can see a very clear "cluster" or tumor in the stimulus.  Here is one where there is a cluster/tumor but it is a little less obvious:

In [ ]:
draw_stimulus(include_source=True, n_background_dots=30*30, n_source_dots=50, source_sd=10, source_border=20, width=400, height=400)

Canvas(height=425, layout=Layout(grid_area='canvas'), width=425)

Finally, here is one where it is pretty hard to tell that there is a cluster at all (oh but there is one involving only 20 dots!):

In [ ]:
draw_stimulus(include_source=True, n_background_dots=30*30, n_source_dots=20, source_sd=10, source_border=20, width=400, height=400)

Canvas(height=425, layout=Layout(grid_area='canvas'), width=425)

Your job in the experiment will be to view stimuli such as the ones above and judge if you think there is a cluster/tumor present in the image.  It can often be hard to tell because even in the random dots there can be little clusters of dots that group together.  Just try to do your best.  Notice that there is a red "absent" button and a green "present" button.  You should press "absent" if you think the cluster/tumor is not there, and press "present" if you think it is there.

Before running the next cell set the subject number to your NYU netid (I will anonymize this later but will let me check that everyone has done the experiment).

## Run the Experiment!

In [ ]:
subject_number = 'tg35' # set your subject number here
exp = Detection_Experiment(subject_number)
exp.start_experiment()

NameError: ignored

## Getting the data

After you run the experiment, the data will be output to a .csv file tied to your subject number.  Also it is available as as pandas data frame in the current kernel environment as `exp.trials`:

In [ ]:
exp.trials.head()


,signal_present,signal_type,trial_num,button_position,subject_number
0,1,60,0,right,tg35
1,1,10,1,right,tg35
2,1,10,2,right,tg35
3,1,15,3,right,tg35
4,0,0,4,right,tg35


In [ ]:
import os

filenames=['../../../shared/sdt-lab/'+f for f in os.listdir('../../../shared/sdt-lab/') if not f.startswith('.')]
filenames
df=pd.concat([pd.read_csv(f) for f in filenames])



In [ ]:
df

,Unnamed: 0,signal_present,signal_type,trial_num,button_position,subject_number,responses,rt,correct_resp
0,0,1,60,0,right,ch3633,1,16455.474,1
1,1,1,10,1,right,ch3633,1,5926.809,1
2,2,1,10,2,right,ch3633,0,3657.883,0
3,3,1,15,3,right,ch3633,0,2311.738,0
4,4,0,0,4,right,ch3633,0,13202.775,1
...,...,...,...,...,...,...,...,...,...
195,195,1,15,195,right,sq627,0,1735.790,0
196,196,0,0,196,right,sq627,0,1695.915,1
197,197,1,60,197,right,sq627,1,1320.913,1
198,198,1,15,198,right,sq627,1,1160.355,1


## Experiment design

In the experiment, you saw a number of trials where field of random colored and positioned dots appear.  On some trials a small "cluster" of points was added to the stimulus.  The number of dots in the cluster was manipulated across conditions so that it was more or less obvious that there was a cluster.  In some cases the number of dots within the cluster was so small that it was hard to detect it, but other cases it might have been more visually obvious.  This is very similar to the type of visual diagnosis that a doctor or airport screener might perform (deciding of a tumor or a gun is present in an image). Your task was to indicate if it was a "stimulus/cluster present" trial or a "stimulus/cluster absent" trial.  The computer recorded your response and your reaction time.

An overview of the situation is shown here:

### The things we manipulated!

There were actually four different types of clusters shown on any given "stimulus present" trial. One was a **small** stimulus made up of 10 dots, one was a medium made up of 15 dots, one type of made up of 25, and finally one was  made up of 60 dots:

<center>
<img src="images/dots.png" width="900">
</center>

As the number of dots increases the general visibility of the cluster becomes more clear and easier to detect.

In total there were 100 stimulus absent trials and 25 of each of the four types of stimulus present trials.  The purpose of this was so that a perfect subject with robot-quality vision would say "yes" half the time in the experiment so there is no overall bias toward one response.

In this lab we are interested in analyzing the effect of strength of the stimulus cluster (i.e., number of dots) on the two signal detection variables we learned about in the previous lecture and lab.

## Analysis Steps


Remember from the previous lab that our model for signal detection looks something like this:
<img src="http://gru.stanford.edu/lib/exe/fetch.php/tutorials/nobias.png?w=&h=&cache=cache">

and we showed how we can compute $d'$ and $c$ by computing only the number of false alarms and hits and subjecting those to a particular mathematical transformation.  In this case we actually have four stimulus conditions (10, 15, 25, or 60).  So our goal will be to compute the hits and false alarms **separately** for the two different types of stimuli.  This will mean that each person in our experiment will be described by eight numbers:

- A d' (d-prime) under (10, 15, 25, 60) visibility cluster conditions
- A c under (10, 15, 25, 60) visibility cluster conditions

Once we have this we want to perform a simple hypothesis test.  Is performance, measured by d' higher in the high visibility conditions?  We have strong reason to suspect this is the case but it'll be a nice check of our understanding if we can related the abstract calculation we performed in the previous lab to answer this simple quesiton.

Answering this question will push our skills at using pandas to organize our data, and using pingouin to compute a t-test.

<div class="alert alert-info" role="alert">
  <strong>Question 1</strong> <br>
    If we compute one $d'$ for each participant for each stimulus conditon and want to compare performance in the two conditions, what type of t-test should we most likely use?
</div>

In this we are to use a double tailed T test so that we can be able to make sure that we have a comparison in the perfomance of the groups
that we are looking at in the experiment that we are performing.

<div class="alert alert-warning" role="alert">
  <strong>Your Answer</strong> <br>
  Enter your plan for the analysis as a markdown cell.
</div>

To conduct this analysis we are to make sure there is the data set that is correct. We ar eto make make the data by making sure we concatinate the different data sets that are given, Then we also make the columns or make a portion of the data so tha twe can be able to use it for the analysis. We are then to perform statistical analysis by making sure we evaluate the data that we have. After making the statistical analysis, we are to make the plots so that we make a comparison like an AB test hypothesis of the data that we are having. After this, we can make conclusion of the data that we are havin by making sure we infer with the analysis that we have made, to be able to come up with the correct analysis and conclusion at the end.

<div class="alert alert-info" role="alert">
  <strong>Question 2</strong> <br>
    To begin we want to read in the data from each participant into a pandas data frame using `pd.read_csv()` and the used `pd.concat()` to combine them.   After you read in the data, inspect the columns and check the data for any inconsistencies that might affect our analysis (e.g., subjects who didn't perform any trials, etc...).
</div>

<div class="alert alert-warning" role="alert">
  <strong>Your Answer</strong> <br>
  Enter your code here.
</div>



In [ ]:
df = pd.read_csv (file_names)
print(df)

As a reminder the following show a simple for loop (called a list comprehension) that lets you get the filenames for the lab data set.  You can use this to find the file names that you need to read in with `pd.read_csv()`.  If you need a reminder refer back to your mental rotation lab which should be on your Jupyterhub node still!

In [ ]:
# this is an example list comprehension which reads in the all the files.
# the f.startswith() part just gets rid of any junk files in that folder
filenames=['sdt1-data/'+f for f in os.listdir('sdt1-data') if not f.startswith('.')]

<div class="alert alert-success" role="alert">
  <strong>Stop and share</strong> <br>
  Did you find any interesting or important points about the data that you want to share with the class?  Let's get everyone on the same page so we get similar results so if you find something important (good or bad) please take a moment and share!
</div>

<div class="alert alert-info" role="alert">
  <strong>Question 3</strong> <br>
    Now that we have the analysis dataframe let's perform our planned analysis on a single subject.  Create a new dataframe, subselecting from the main frame with the data from a single subject.
</div>

<div class="alert alert-warning" role="alert">
  <strong>Your Answer</strong> <br>
  Enter your code here.
</div>

In [ ]:
df.loc[0:3]

<div class="alert alert-info" role="alert">
  <strong>Question 4</strong> <br>
    Referring back to the first notebook from the lab compute the hits and false alarms for this subject for each stimulus present condition.  One note about the false alarms: remember that false alarms are trials where the stimulus was absent but the subject said it was present.  There is only one type of these trials in the experiment.  So when you compute the statistics you really will create five numbers of this participant.  The hits for each of the stimulus present conditions, and an overall false alarm rate.  What are these numbers for this subject?  Remember also that if you take the `mean()` of a column of 0/1 values it will tell you the proportion of those trials which are 1.
</div>

<div class="alert alert-warning" role="alert">
  <strong>Your Answer</strong> <br>
  Enter your code here.
</div>

In [ ]:
# generate data sample
data = rand(1000)
# calculate quartiles
quartiles = percentile(data, [25, 50, 75])
# calculate min/max
data_min, data_max = data.min(), data.max()
# print 5-number summary
print('Min: %.3f' % data_min)
print('Q1: %.3f' % quartiles[0])
print('Median: %.3f' % quartiles[1])
print('Q3: %.3f' % quartiles[2])
print('Max: %.3f' % data_max)

<div class="alert alert-info" role="alert">
  <strong>Question 5</strong> <br>
    Ok now compute the d' and c values for this subject.  Refer back to the previous notebook for reference if you need the equations and python code.  What can you tell from looking at your computed values?  If for some reason you can't compute these values in your experiment (e.g., if the subject has no hits or no false alarms) then try question 4 again with a different subject.
</div>

<div class="alert alert-warning" role="alert">
  <strong>Your Answer</strong> <br>
  Enter your code here.
</div>

In [ ]:
def helper(n):
   p = n
   i = 2
   while i * i <= n:
      if n % i == 0:
         p -= p // i
      while n % i == 0:
         n = n // i
      if i != 2:
         i += 2
      else:
         i += 1
   if n > 1:
      p -= p // n
   return p

def solve(a, b, c, d, n):
   if b == 0 or (c == 0 and d != 0):
      return pow(a, 0, n)
   if c == 1 or d == 0:
      return pow(a, b, n)
   if a == 0 or a % n == 0:
      return 0
   if d == 1:
      return pow(a, b * c, n)
   p = helper(n)
   e = pow(c, d, p) + p
   return pow(pow(a, b, n), e, n)

print(solve(2, 3, 2, 4, 10))

6


<div class="alert alert-info" role="alert">
  <strong>Question 6</strong> <br>
    Ok, so now you have performed the desired analysis for one subject.  You have computed eight numbers for that subject described above.  Now we want to repeat this for each subject. There are two ways to do this.  One is using the pandas groupby() function.  This is somewhat of the advanced mode version but if you do this you can probably perform the analysis in only 5-6 lines of code.  An alternative approach is to generalize the code you just wrote to use a for loop that iterates over each subject in the data frame.  Which ever way you choose you might find it helpful to refer to the <a href="http://gureckislab.org/courses/fall19/labincp/tips/fortyforloops.html">Forty For Loops</a> notebook I provided which shows some example templates for using for loops to analyze individual subject data in pandas dataframes.  The end result is that you want a dataframe containing the d' and c' for each subject for each stimulus condition.
</div>

<div class="alert alert-warning" role="alert">
  <strong>Your Answer</strong> <br>
  Enter your code here.
</div>

In [ ]:
## Perform a 2 T test


import scipy.stats as stats

# Creating data groups
data_group1 = np.array([14, 15, 15, 16, 13, 8, 14,
                        17, 16, 14, 19, 20, 21, 15,
                        15, 16, 16, 13, 14, 12])

data_group2 = np.array([15, 17, 14, 17, 14, 8, 12,
                        19, 19, 14, 17, 22, 24, 16,
                        13, 16, 13, 18, 15, 13])

# Perform the two sample t-test with equal variances
stats.ttest_ind(a=data_group1, b=data_group2, equal_var=True)

Ttest_indResult(statistic=-0.6337397070250238, pvalue=0.5300471010405257)

In [ ]:
# Python program to conduct two-sample
# T-test using pingouin library

# Importing library
from statsmodels.stats.weightstats import ttest_ind
import numpy as np
import pingouin as pg

# Creating data groups
data_group1 = np.array([160, 150, 160, 156.12, 163.24,
                        160.56, 168.56, 174.12,
                        167.123, 165.12])
data_group2 = np.array([157.97, 146, 140.2, 170.15,
                        167.34, 176.123, 162.35, 159.123,
                        169.43, 148.123])

# Conducting two-sample ttest
result = pg.ttest(data_group1,
                  data_group2,
                  correction=True)

# Print the result
print(result)


               T        dof alternative     p-val           CI95%   cohen-d  \
T-test  0.653148  14.389477   two-sided  0.523961  [-6.38, 11.99]  0.292097   

         BF10     power  
T-test  0.462  0.094912  


<div class="alert alert-info" role="alert">
  <strong>Question 7</strong> <br>
    Now that you have your summary statistics computed, our next step is to ask if the conditions are different on either the d' or c measure.  For now concentrate on the easiest and hardest conditions to see if there is a effect in the most dissimilar conditions.  Then compare the hardest and second hardest conditions. Conduct the appropriate t-test to tell if these conditions are different.  The chapter reading from this week has all the code you need.  However you might find it helpful to refer to the pingouin documentation for the <a href="https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest">t-test function</a>.
</div>

<div class="alert alert-warning" role="alert">
  <strong>Your Answer</strong> <br>
  Enter your code here.
</div>

In [ ]:
# Python program to conduct
# two-sample t-test using statsmodels

# Importing library
from statsmodels.stats.weightstats import ttest_ind
import numpy as np
import pingouin as pg

# Creating data groups
data_group1 = np.array([160, 150, 160, 156.12,
                        163.24,
                        160.56, 168.56, 174.12,
                        167.123, 165.12])
data_group2 = np.array([157.97, 146, 140.2, 170.15,
                        167.34, 176.123, 162.35,
                        159.123, 169.43, 148.123])

# Conducting two-sample ttest
ttest_ind(data_group1, data_group2)

(0.6531479162158739, 0.5219170107019715, 18.0)

<div class="alert alert-info" role="alert">
  <strong>Question 8 (BONUS)</strong> <br>
    This question is a bonus but asks you to verify the assumptions of the chosen t-test hold.  Usually this involves looking to see that the distribution of scores you are analyzing are roughly "normally" distributed.  The textbook showed a few example way to check including the histogram, the qqplot, and the Shapiro-Wilks test.  Perform this test on your data.  If the data seem very non-normal you might consider switching your t-test to the non-parametric alternative discussed in the chapter (the wilcoxon or Mahn-Whitney test).
</div>

<div class="alert alert-warning" role="alert">
  <strong>Your Answer</strong> <br>
  Enter your code here.
</div>

Regarding the scale of measurement of the data plots that we are having in this case, whether there is a random sampling, normality of data distribution that is the plots of data that we have, adequacy of sample size, and equality of variance in standard deviation.

<div class="alert alert-info" role="alert">
  <strong>Question 9</strong> <br>
  Ok great job!  You have your data, you have your t-test, you have everything you need to answer the following questions: 1. Is performance higher in the low number of dots or high number of dots condition?  How do you know?  How would you report the statistical evidence in favor of your conclusion in a paper?  Is the criterion different between the conditions?  How do you know?  Overall was performance high or low in this task?
</div>

<div class="alert alert-warning" role="alert">
  <strong>Your Answer</strong> <br>
  Enter your Markdown text here.
</div>

In the analysis we have done it is quite clear to us that we have:
1. The performance in the lower number of dots. HTis is known as we have the visuaoazaton of the plotss that are in the adta that we hhave. the statistical evidene of this is dome by plotting of teh AB test of hypothesis data plot. THis is clear by the diagram that is shown above as we draw the conclusion of the data from the statisical analysis of the data.
the cretirion that us used is a little different when we are to plot on the codnitions that are relevant to the ones that are there in the data that is given.
Thi is eveident by the D' calculation of the statisctic. It is actually different in the hypothesis test taht is conducted in every statisctical package that are given.
And the perfomance overal in this was low, as compared to the one of the statistical data as we look at the one for the AB analysis.

<div class="alert alert-info" role="alert">
  <strong>Question 10</strong> <br>
  The final question asks you to consider limitations to this experiment design.  What other analyses might be interesting to perform on this data?  If you could change the experiment is there another type of question you would ask?
</div>

<div class="alert alert-warning" role="alert">
  <strong>Your Answer</strong> <br>
  Enter your Markdown text here.
</div>

In this experiment, the consideration of the points of the data could be to make sure that there is an AB testing so that we are able to
make a better comparison of the data. Also it is also important to make sure that we have proper hypothesis testing.